Train a simple deep CNN on the CIFAR10 small images dataset.

It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs, though it is still underfitting at that point.

In [ ]:
# Train a simple deep CNN on the CIFAR10 small images dataset.
library(keras)
use_condaenv("r-tensorflow")
use_session_with_seed(7)
options(keras.view_metrics = TRUE)

In [ ]:
# Parameters --------------------------------------------------------------
# It takes ~4 minutes to run each epoch, adjust the number to match time left
batch_size <- 32
epochs <- 2
data_augmentation <- TRUE

In [ ]:
# Data Preparation --------------------------------------------------------

# See ?dataset_cifar10 for more info
cifar10 <- dataset_cifar10()

# Scale RGB values in test and train images, convert to float 32 first, then divide by 255
x_train <- 
x_test <- 

# convert the labels to categorical data with 10 classes
y_train <- 
y_test <- 

In [ ]:
# Defining Model ----------------------------------------------------------

# Initialize sequential model
model <- 

model %>%

# Start with hidden 2D convolutional layer being fed 32x32 pixel images 
# We need 32 filters, a 3x3 kernel, padding and input shape.
# This is followed by an activation layer.
layer_conv_2d(     ) %>%
layer_activation() %>%

# Second hidden layer, as above but only need filter and kernel size, followed by activation layer
layer_conv_2d() %>%
layer_activation() %>%

# Use max pooling (2x2) and dropout (0.25)
layer_max_pooling_2d() %>%
layer_dropout() %>%

# 2 additional hidden 2D convolutional and activation layer sets 
# as above, however no more input shape required
layer_conv_2d() %>%
layer_activation() %>%
layer_conv_2d() %>%
layer_activation() %>%

# Use max pooling and dropout once more, same as above
layer_max_pooling_2d() %>%
layer_dropout() %>%

# Flatten max filtered output into feature vector 
# feed into dense layer (512), followed by an activation and droput layer (0.5)
layer_flatten() %>%
layer_dense() %>%
layer_activation() %>%
layer_dropout() %>%

# Outputs from dense layer are projected onto 10 unit output layer
# plus one last activation layer (softmax)
layer_dense() %>%
layer_activation()

opt <- optimizer_rmsprop(lr = 0.0001, decay = 1e-6)

model %>% compile(
loss = "categorical_crossentropy",
optimizer = opt,
metrics = "accuracy"
)

In [ ]:
# Training ----------------------------------------------------------------

if(!data_augmentation){
  
  history <- model %>% fit(
    x_train, y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = list(x_test, y_test),
    shuffle = TRUE
  )
  
} else {
  
  datagen <- image_data_generator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = TRUE
  )
  
  datagen %>% fit_image_data_generator(x_train)
  
  history <- model %>% fit_generator(
    flow_images_from_data(x_train, y_train, datagen, batch_size = batch_size),
    steps_per_epoch = as.integer(50000/batch_size), 
    epochs = epochs, 
    validation_data = list(x_test, y_test)
  )
  
}

In [ ]:
plot(history)